In [57]:
'''
%pip install xgboost scikit-learn
'''

'\n%pip install xgboost scikit-learn\n'

In [58]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_auc_score,roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


In [59]:
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
data=pd.DataFrame(pd.concat([breast_cancer_wisconsin_original.data.features,breast_cancer_wisconsin_original.data.targets],axis=1))

In [60]:
#Tratamiento de los valores nulos
data_nul=data.dropna(how='any')
X_nul=data_nul.drop(['Bare_nuclei'],axis=1)
y_nul=data_nul['Bare_nuclei']
X_train_nul,X_test_nul,y_train_nul,y_test_nul=train_test_split(X_nul,y_nul,test_size=.25,random_state=40,stratify=y_nul)

In [61]:
#RandomForest
params={
    'max_depth':[2,4,6,8,10,12],
    'criterion':['gini','entropy','log_loss']
}
clf_nul=RandomForestClassifier(n_estimators=100,random_state=40)
grid_nul=GridSearchCV(estimator=clf_nul,param_grid=params,cv=4)
grid_nul.fit(X_train_nul,y_train_nul)
best_nul=grid_nul.best_estimator_

print('Accuracy train: ',accuracy_score(y_train_nul,best_nul.predict(X_train_nul)))
print('Accuracy test: ',accuracy_score(y_test_nul,best_nul.predict(X_test_nul)))

/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


Accuracy train:  0.75390625
Accuracy test:  0.7602339181286549


In [62]:
#KNeighborsClassifier
param_grid = {
    'n_neighbors': [3, 5, 7,9,10,12],   # Número de vecinos a considerar
    'p': [1, 2]  # Parámetro p para la distancia (1 para Manhattan, 2 para Euclidiana)
}

# Crea el KNeighborsClassifier
knn_classifier = KNeighborsClassifier()

# Crea el objeto GridSearchCV
grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Entrena el modelo con la búsqueda de hiperparámetros
grid_search.fit(X_train_nul, y_train_nul)
best_knn_classifier = grid_search.best_estimator_
y_pred = best_knn_classifier.predict(X_test_nul)

# Calcula la precisión del mejor modelo
accuracy = accuracy_score(y_test_nul, y_pred)
print("Accuracy test:", accuracy)
print('Accuracy train: ',accuracy_score(y_train_nul,best_knn_classifier.predict(X_train_nul)))



/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Accuracy test: 0.7602339181286549
Accuracy train:  0.748046875


In [63]:

# Define los hiperparámetros que deseas buscar
param_grid = {
    'C': [0.1, 1, 10],  # Parámetro de regularización
    'kernel': ['linear', 'rbf', 'poly'],  # Tipo de kernel
    'gamma': ['scale', 'auto']  # Coeficiente del kernel para 'rbf' y 'poly'
}

# Crea el clasificador SVM
svm_classifier = SVC()

# Crea el objeto GridSearchCV
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Entrena el modelo con la búsqueda de hiperparámetros
grid_search.fit(X_train_nul, y_train_nul)

# Haz predicciones con el mejor modelo
best_svm_classifier = grid_search.best_estimator_
y_pred = best_svm_classifier.predict(X_test_nul)

# Calcula la precisión del mejor modelo
accuracy = accuracy_score(y_test_nul, y_pred)
print("Accuracy test:", accuracy)
print('Accuracy train: ',accuracy_score(y_train_nul,best_svm_classifier.predict(X_train_nul)))



/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Accuracy test: 0.7426900584795322
Accuracy train:  0.767578125


In [64]:
#Rellenamos usando KNN
rellenar=data[data['Bare_nuclei'].isnull()]
preds=best_knn_classifier.predict(rellenar.drop('Bare_nuclei',axis=1))
data.loc[data['Bare_nuclei'].isnull(), 'Bare_nuclei'] = preds
data.isna().sum()

Clump_thickness                0
Uniformity_of_cell_size        0
Uniformity_of_cell_shape       0
Marginal_adhesion              0
Single_epithelial_cell_size    0
Bare_nuclei                    0
Bland_chromatin                0
Normal_nucleoli                0
Mitoses                        0
Class                          0
dtype: int64

In [65]:
#Eliminacion de duplicados
#data.drop_duplicates(subset=data.columns,inplace=True)

In [66]:
X=data.drop('Class',axis=1)
y=data['Class'].apply(lambda x: 0 if x==2 else 1)

In [67]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=70,stratify=y)
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [68]:
clf=RandomForestClassifier(n_estimators=20,random_state=40,n_jobs=4)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=20, n_jobs=4, random_state=40)

In [69]:
pred=clf.predict(X_test)
print(accuracy_score(y_train,clf.predict(X_train)))
print(accuracy_score(y_test,pred))
print(precision_score(y_test,pred))

1.0
0.9666666666666667
0.922077922077922


In [70]:
print(confusion_matrix(y_test,pred))

[[132   6]
 [  1  71]]


In [71]:
params={
    'n_estimators':[10,15,20,30,40,50],
    'max_depth':[3,4,6,8,10,12]
}
grid=GridSearchCV(estimator=clf,param_grid=params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)
grid.best_score_

0.979549758047549

In [72]:
mod=grid.best_estimator_

In [73]:
mod.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 15,
 'n_jobs': 4,
 'oob_score': False,
 'random_state': 40,
 'verbose': 0,
 'warm_start': False}

In [74]:
print(accuracy_score(y_train,mod.predict(X_train)))
print(accuracy_score(y_test,pred))
print(precision_score(y_test,pred))

0.9979550102249489
0.9666666666666667
0.922077922077922
